<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/main/HomeWork_less8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [203]:
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [204]:
!pip install pymorphy2 stop_words slovnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [285]:
import numpy as np
import pandas as pd
import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split


import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping  
from tensorflow.keras.optimizers import Adam

# Обработка

In [372]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 7/отзывы за лето.xls")
df

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01
20655,5,Ок,2017-06-01
20656,4,Доволен,2017-06-01
20657,1,"Песопаснасть, рут ни нужын",2017-06-01


In [373]:
max_words = 10000
max_len = 30
num_classes = 1

In [374]:
# обработем текст + разделим таргет с мультикласса на оценки 1-3 отрицательные с 4-5 положительные
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df['target'] = (df['Rating'] > 3)*1

In [375]:
df['target'] = df['target'].astype(int)
df.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично всё,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [376]:
df.drop(['Content', 'Date'], axis=1, inplace=True)

In [377]:
train_corpus = " ".join(df["text"])
train_corpus = train_corpus.lower()

In [378]:
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [379]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [380]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [381]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [382]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.3,
                                                    random_state=11, stratify=df['target'])

In [383]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [384]:
X_train.shape

(14461, 30)

In [336]:
# num_classes = 2
# y_train = to_categorical(y_train, num_classes)
# y_test = to_categorical(y_test, num_classes)

# Модели

In [280]:
# Переменная в которой будем хранить данные с разных моделей
result_history = []

## CNN

In [270]:
# Training
epochs = 5
batch_size = 512

In [277]:
model = Sequential()
model.add(Embedding(
                    input_dim=max_words, 
                    output_dim=256, 
                    input_length=max_len, 
                    trainable=True,
                    mask_zero=True))
model.add(Conv1D(256, 3))
model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3))
model.add(MaxPooling1D(2))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Dropout(0.3))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [278]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [279]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test],
                    callbacks=[tensorboard])

Epoch 1/5
29/29 [==============================] - 31s 1s/step - loss: 0.3714 - accuracy: 0.8324 - val_loss: 0.2576 - val_accuracy: 0.8911
Epoch 2/5
29/29 [==============================] - 30s 1s/step - loss: 0.2129 - accuracy: 0.9128 - val_loss: 0.2260 - val_accuracy: 0.9064
Epoch 3/5
29/29 [==============================] - 31s 1s/step - loss: 0.1563 - accuracy: 0.9396 - val_loss: 0.2212 - val_accuracy: 0.9040
Epoch 4/5
29/29 [==============================] - 30s 1s/step - loss: 0.1166 - accuracy: 0.9570 - val_loss: 0.2647 - val_accuracy: 0.8987
Epoch 5/5
29/29 [==============================] - 30s 1s/step - loss: 0.0813 - accuracy: 0.9728 - val_loss: 0.2862 - val_accuracy: 0.8963


In [281]:
result_history.append(['CNN default', history.history.get('val_accuracy')[-1]])

## RNN

In [406]:
# Training
epochs = 4
batch_size = 512

**RNN**

In [409]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(SimpleRNN(126, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test]
                    )

Epoch 1/4
29/29 [==============================] - 10s 284ms/step - loss: 0.5758 - accuracy: 0.7975 - val_loss: 0.3636 - val_accuracy: 0.8800
Epoch 2/4
29/29 [==============================] - 7s 245ms/step - loss: 0.3189 - accuracy: 0.8863 - val_loss: 0.2639 - val_accuracy: 0.8922
Epoch 3/4
29/29 [==============================] - 7s 244ms/step - loss: 0.2424 - accuracy: 0.9084 - val_loss: 0.2239 - val_accuracy: 0.9079
Epoch 4/4
29/29 [==============================] - 7s 245ms/step - loss: 0.2082 - accuracy: 0.9264 - val_loss: 0.2130 - val_accuracy: 0.9164


In [410]:
result_history.append(['RNN', history.history.get('val_accuracy')[-1]])

**LSTM**

In [407]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(LSTM(126, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test]
                    )

Epoch 1/4
29/29 [==============================] - 24s 703ms/step - loss: 0.5538 - accuracy: 0.8069 - val_loss: 0.3437 - val_accuracy: 0.8637
Epoch 2/4
29/29 [==============================] - 20s 684ms/step - loss: 0.2951 - accuracy: 0.8916 - val_loss: 0.2447 - val_accuracy: 0.8971
Epoch 3/4
29/29 [==============================] - 20s 683ms/step - loss: 0.2235 - accuracy: 0.9148 - val_loss: 0.2162 - val_accuracy: 0.9169
Epoch 4/4
29/29 [==============================] - 20s 686ms/step - loss: 0.1867 - accuracy: 0.9316 - val_loss: 0.2202 - val_accuracy: 0.9145


In [408]:
result_history.append(['LSTM', history.history.get('val_accuracy')[-1]])

**GRU**

In [411]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(GRU(126, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test]
                    )

Epoch 1/4
29/29 [==============================] - 20s 586ms/step - loss: 0.5519 - accuracy: 0.8010 - val_loss: 0.3721 - val_accuracy: 0.8324
Epoch 2/4
29/29 [==============================] - 16s 567ms/step - loss: 0.2964 - accuracy: 0.8779 - val_loss: 0.2254 - val_accuracy: 0.9113
Epoch 3/4
29/29 [==============================] - 16s 567ms/step - loss: 0.2115 - accuracy: 0.9205 - val_loss: 0.2192 - val_accuracy: 0.9114
Epoch 4/4
29/29 [==============================] - 18s 620ms/step - loss: 0.1737 - accuracy: 0.9401 - val_loss: 0.2506 - val_accuracy: 0.9056


In [412]:
result_history.append(['GRU', history.history.get('val_accuracy')[-1]])

## CNN + LSTM

In [423]:
# Training
epochs = 4
batch_size = 512

In [424]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(256, 3))
model.add(MaxPooling1D(2))
model.add(GRU(126, recurrent_dropout=0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=[X_test, y_test]
                    )

Epoch 1/4
29/29 [==============================] - 19s 559ms/step - loss: 0.4184 - accuracy: 0.8179 - val_loss: 0.2996 - val_accuracy: 0.8701
Epoch 2/4
29/29 [==============================] - 16s 541ms/step - loss: 0.2716 - accuracy: 0.8853 - val_loss: 0.2304 - val_accuracy: 0.8921
Epoch 3/4
29/29 [==============================] - 15s 536ms/step - loss: 0.2011 - accuracy: 0.9203 - val_loss: 0.2305 - val_accuracy: 0.9074
Epoch 4/4
29/29 [==============================] - 16s 540ms/step - loss: 0.1529 - accuracy: 0.9432 - val_loss: 0.2541 - val_accuracy: 0.9048


In [425]:
result_history.append(['CNN + LSTM', history.history.get('val_accuracy')[-1]])

# Вывод

In [427]:
# Соберем данные в пандасе
result_df = pd.DataFrame({'Name model': [result_history[i][0] for i in range(len(result_history))], 'Validation accuracy': [result_history[i][1] for i in range(len(result_history))]})

In [429]:
result_df.sort_values(by='Validation accuracy', ascending=False)

,Name model,Validation accuracy
2,RNN,0.916425
1,LSTM,0.914489
3,GRU,0.905615
4,CNN + LSTM,0.904808
0,RNN default,0.896257


Как видим из результата RNN и LSTM показала наилучшее качество, CNN + LSTM	отработала хуже, в целом столнкулся с проблемой того что идет переобучение на трейне, и выше 91.7 результата не смог получить